# 最基本查询示例 v0.2

In [ ]:
初步结论：

- 本示例主要是用documentSummaryIndex, 效果较好
- 适用于300篇左右文章的查询
- 索引保存在本地文件中

参考：

- https://note.com/pescafe/n/nba8306b88344

## 准备

In [1]:
%%time

# 加载llm和embeddings
%run ../utils.py

CPU times: user 3.28 s, sys: 344 ms, total: 3.63 s
Wall time: 3.31 s


## 加载新闻数据

In [2]:
%%time

import os
import json

directory = './news'

news_list=[]

# List all files in the specified directory
for file in os.listdir(directory):
    if file.endswith('.json'):
        file_path = os.path.join(directory, file)
        # Open and read the JSON file
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                news_list.append(data)
            except json.JSONDecodeError as e:
                print(f"Error reading {file_path}: {e}")

len(news_list)

CPU times: user 1.15 ms, sys: 63 µs, total: 1.21 ms
Wall time: 903 µs


3

## 创建 Document

In [3]:
%%time

from llama_index.core import Document

documents = [Document(text=t['content'], 
                      metadata={"title": t['title'],
                                'publish_time': t['publish_time'],
                                'author': t['author'],
                                'images': t['images'],
                               }) 
             for t in news_list]

for document in documents:
    document.doc_id = document.metadata["title"]

CPU times: user 203 µs, sys: 0 ns, total: 203 µs
Wall time: 178 µs


## 文档摘要索引

### 生成文档摘要索引

In [4]:
%%time

import nest_asyncio
nest_asyncio.apply()

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/3 [00:00<?, ?it/s]

current doc id: 中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明
current doc id: 鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家
current doc id: 德国海外商会联盟发布报告:在华德企最担心价格压力,欧盟加征关税只会适得其反


Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 463 ms, sys: 11.5 ms, total: 475 ms
Wall time: 55.3 s


### 保存文档摘要索引

In [5]:
%%time

doc_summary_index.storage_context.persist("index")

CPU times: user 13.1 ms, sys: 102 µs, total: 13.2 ms
Wall time: 12.5 ms


### 加载文档摘要索引

In [6]:
%%time

from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 20.7 ms, sys: 445 µs, total: 21.2 ms
Wall time: 20.7 ms


In [12]:
%%time

storage_context.docstore.get_all_ref_doc_info()

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 17.2 µs


{'中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明': RefDocInfo(node_ids=['98d21f17-e58d-481e-b72b-3334c1760e75', '95e2eab9-aca6-4989-9123-071a62600954', 'c30c2364-e8b5-49d2-a198-866ba581c6a9'], metadata={'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', 'publish_time': '2024-06-18 15:34:33', 'author': '林琛力', 'images': []}),
 '鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家': RefDocInfo(node_ids=['cf96de48-cf2f-43f9-b812-50d840418b4a', '92221d48-c598-4b68-a66a-d8a4bf09e414', '27ea40de-aa54-4e9c-822f-269b1addf075', '01203e0e-e0fd-41b9-9cad-95b4c08ba1bd', '0795f1a7-364d-42e3-a24b-339196737961', '222a1f31-38f5-408b-8bb4-c7bde065776c', '337c2eae-27c0-4158-a73d-8c41deef30fe', 'd3652456-3756-4311-b12d-53f9344b3a6d', 'c6bdf2f3-11a9-4bba-bfc5-7a459d5061bf', '1bab38e7-2d4b-4e22-af55-ce58e4a92566', '2f618c47-fa62-4e0d-9b63-26211a506a29', 'bc704ad4-b6bd-4ce8-89da-523c5e72aebe'], metadata={'title': '鲍韶山:中澳正在消除分歧,但澳大利亚仍是一个焦虑的国家', 'publish_time': '2024-06-17 07:43:21', 'author': '苏堤', 'images': ['https://i.guancha.cn/news/external/2024/0

In [16]:
%%time

storage_context.docstore.get_nodes(['98d21f17-e58d-481e-b72b-3334c1760e75'])[0].text

CPU times: user 125 µs, sys: 12 µs, total: 137 µs
Wall time: 140 µs


'据中国驻菲律宾大使馆微信公众号消息,中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明。\n一、问:近日,英国路透社报道披露,新冠疫情期间美国军方为了遏制中国影响力,发动秘密心理战,使用社交媒体账号假冒菲律宾人散播中国疫苗和抗疫物资不安全的虚假信息,煽动菲民众对中国疫苗的不信任和恐惧,导致菲成为东南亚接种率最低、死亡率最高国家之一,数以万计民众染疫死亡。菲国内出现要求菲政府和国会对此展开调查的呼声。使馆对此有何评论?'

In [18]:
storage_context.docstore.get_nodes(['c30c2364-e8b5-49d2-a198-866ba581c6a9'])[0].text

"The provided text discusses a statement made by the Chinese Embassy in the Philippines regarding allegations of secret psychological warfare conducted by the US military during the COVID-19 pandemic to discredit Chinese vaccines. The statement addresses two main points:\n\n1. **US Military's Alleged Actions**: It accuses the US military of using social media accounts to impersonate Filipinos and spread false information about the safety of Chinese vaccines, thereby sowing distrust among Filipino citizens towards Chinese vaccines. This is said to have contributed to low vaccination rates and high death tolls in the Philippines during the pandemic.\n\n2. **International Repercussions**: The text mentions that this issue has sparked calls for investigation by both the Philippine government and its parliament. It also highlights the frustration of Filipinos who are demanding accountability from their leaders regarding the situation.\n\nThe statement further criticizes US actions, emphasiz

In [7]:
doc_summary_index.get_document_summary("中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明")

"The provided text discusses a statement made by the Chinese Embassy in the Philippines regarding allegations of secret psychological warfare conducted by the US military during the COVID-19 pandemic to discredit Chinese vaccines. The statement addresses two main points:\n\n1. **US Military's Alleged Actions**: It accuses the US military of spreading false information about the safety and efficacy of Chinese vaccines through social media, aiming to undermine trust in Chinese vaccines among Filipino citizens. This is said to have contributed to low vaccination rates and high death tolls in the Philippines during the pandemic.\n\n2. **International Response**: The text mentions that there are calls for investigations by the Philippine government and its legislature on this matter. It also highlights previous revelations about similar actions taken by other countries, which have now been substantiated through a recent report.\n\nThe statement expresses strong disapproval of these actions,

### 使用文档摘要索引查询

#### 高级查询 api - 基于嵌入的召回

In [20]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

response = query_engine.query("中国驻菲律宾大使馆对美国军方的行动有何评论？")
response.print_response_stream()

根据提供的信息，没有直接提及中国驻菲律宾大使馆对美国军方行动的具体评论。所给的信息主要关注了澳大利亚与中国之间的经济、外交关系以及安全态势，并讨论了供应链风险、贸易伙伴份额和政策影响等议题。关于中国驻菲律宾大使馆的评论或声明并未在这些资料中出现。

要获取有关中国驻菲律宾大使馆针对美国军方行动的官方立场或评论，通常需要查阅最新的外交公告、新闻发布会记录或者直接访问中国外交部或相关使领馆的官方网站以获得最准确和权威的信息。CPU times: user 252 ms, sys: 15.2 ms, total: 267 ms
Wall time: 35.1 s


In [21]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

response = query_engine.query("澳大利亚人对中国的恐惧再次被激活是怎么回事？")
response.print_response_stream()

澳大利亚人对中国的恐惧再次被激活，这一现象主要与以下几个因素相关：

1. **地缘政治和经济关系**：受到美国“去风险”和“脱钩”政策的影响，澳大利亚的外交政策在处理与中国的关系时受到了外部压力。这表明澳大利亚在平衡其经济利益、国家安全以及与美国的盟友关系之间存在复杂考量。

2. **供应链安全与产能发展**：澳大利亚寻求提高关键矿产和稀土等资源的附加值，并在这一过程中依赖中国企业的参与。然而，如果将这些关键资源的供应链与美国的安全需求挂钩，可能会导致产能发展的限制风险。这反映了澳大利亚在确保经济独立性、国家安全以及全球市场竞争力之间的挑战。

3. **历史记忆**：19世纪60年代针对澳大利亚金矿华工的暴力事件留下了深刻的历史烙印，在某种程度上影响了当前对中国的看法和态度。这种历史背景可能加深了公众对中国形象的认知，尤其是与早期移民政策相关的负面情绪。

4. **政策自主性与外交依赖**：过去20年里，澳大利亚国防政策强调自给自足，但这一策略不再适用。国家主权在国防能力中的从属地位使得澳大利亚的外交决策受到外部力量的影响，特别是美国的安全利益和全球主导地位。这表明澳大利亚在处理与中国的关系时需要平衡其国内安全、经济需求与国际盟友之间的关系。

5. **焦虑与依赖**：澳大利亚被描述为一个“焦虑的国家”，其对亚洲尤其是对中国的态度，在很大程度上取决于美国的一举一动。这种外交政策上的依赖性导致了公众对中国持有高度警惕和不信任的态度，尤其是在国家安全和战略利益方面。

综上所述，澳大利亚人对中国的恐惧再次被激活是由于复杂的国际关系、历史因素、政策考量以及地缘政治环境的相互作用所导致的结果。这一现象反映了澳大利亚在处理与中国的关系时面临的挑战和不确定性。CPU times: user 580 ms, sys: 21.9 ms, total: 602 ms
Wall time: 1min


In [14]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

response = query_engine.query("中国外交部发言人汪文斌是怎么介绍中国新能源产业的发展的？")
response.print_response_stream()

中国外交部发言人汪文斌表示，中国新能源产业的发展是基于企业长期的技术积累和开放竞争的结果。在充分的竞争环境中，通过技术实力拼争获得市场地位，而不是依赖于政府补贴。这一发展过程完全遵循了市场经济规律和公平竞争的原则。中国认为其新能源产能是推动全球经济绿色转型的先进力量，并非过剩产能。

要获取汪文斌关于中国新能源产业发展的具体介绍内容，建议查阅官方声明、新闻发布会记录或演讲稿等官方渠道发布的资料。通常情况下，这些信息会通过政府官方网站、外交部新闻发布平台或是权威媒体进行发布和传播。CPU times: user 223 ms, sys: 16.7 ms, total: 240 ms
Wall time: 31.2 s


#### 底层召回api - 基于嵌入的召回

In [22]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=5,
)

from llama_index.core.query_engine import RetrieverQueryEngine

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query("中国驻菲律宾大使馆对美国军方的行动有何评论？")
response.print_response_stream()


根据提供的信息，没有直接提及中国驻菲律宾大使馆对美国军方行动的具体评论。所给的信息主要关注了中澳关系、澳大利亚与中国之间的经济联系以及澳大利亚在面对中美竞争时的“焦虑”情绪和对外交政策的影响。因此，无法从这些资料中提取出关于中国驻菲律宾大使馆针对美国军方行动的评论内容。

要获取有关此问题的答案，需要查阅最新的外交声明或官方文件，这些通常由相关政府机构或大使馆发布。CPU times: user 201 ms, sys: 11.9 ms, total: 213 ms
Wall time: 33.4 s


#### 底层召回api - 基于llm的召回

In [10]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
    # choice_select_prompt=None,
    # choice_batch_size=10,
    # choice_top_k=1,
    # format_node_batch_fn=None,
    # parse_choice_select_answer_fn=None,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True
)

# query
response = query_engine.query("中国驻菲律宾大使馆对美国军方的行动有何评论？")
response.print_response_stream()

中国驻菲律宾大使馆发言人表示，对于美军在新冠疫情期间发动的秘密心理战行为深感愤慨。美国声称尊重人权，但在关乎菲律宾人民生命健康的最大人权问题上采取了相反的做法。美国在关键时刻让菲律宾人民孤立无援，并出于地缘政治私利阻止菲律宾获得中国援助。针对全球公共卫生的重大问题，美国散布虚假信息和欺骗行为被认为是不可接受的。

发言人强调，如果美国真正遵循其言论，就应该停止炮制针对别国的虚假信息，切实承担大国责任，尊重地区国家维护民众健康福祉和地区和平稳定的努力。同时，中菲两国人民都应对美国军方在网上继续进行的秘密信息行动保持高度警惕，并及时揭露和抵制可能影响南海和平稳定的煽风拱火行为。

大使馆呼吁中菲关系和南海地区的和平与稳定由双方共同呵护，而不是被外部势力所塑造或破坏。CPU times: user 279 ms, sys: 45.9 ms, total: 324 ms
Wall time: 27.4 s


### 直接使用文档摘要索引召回

In [23]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve("中国外交部发言人汪文斌是怎么介绍中国新能源产业的发展的？")

len(retrieved_nodes)

CPU times: user 5.54 ms, sys: 278 µs, total: 5.82 ms
Wall time: 111 ms


3

In [25]:
retrieved_nodes[0]

NodeWithScore(node=TextNode(id_='f361e574-452f-4d2c-aa0a-056580589ffd', embedding=None, metadata={'title': '德国海外商会联盟发布报告:在华德企最担心价格压力,欧盟加征关税只会适得其反', 'publish_time': '2024-06-18 15:28:10', 'author': '陈思佳', 'images': ['https://i.guancha.cn/news/external/2024/06/18/20240618144107163.jpg', 'https://i.guancha.cn/news/external/2024/06/18/20240618144205462.jpg']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='德国海外商会联盟发布报告:在华德企最担心价格压力,欧盟加征关税只会适得其反', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '德国海外商会联盟发布报告:在华德企最担心价格压力,欧盟加征关税只会适得其反', 'publish_time': '2024-06-18 15:28:10', 'author': '陈思佳', 'images': ['https://i.guancha.cn/news/external/2024/06/18/20240618144107163.jpg', 'https://i.guancha.cn/news/external/2024/06/18/20240618144205462.jpg']}, hash='f6c16ef1e2793c53a61a9106ef4d181167ba5b1c014ab2f0c25c1d2e5f5f731a'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='bc3f68f6-037e-4b48-a88a-39e

In [26]:
%%time

retrieved_nodes = retriever.retrieve("中国驻菲律宾大使馆对美国军方的行动有何评论？")
retrieved_nodes[0]

CPU times: user 5.43 ms, sys: 145 µs, total: 5.58 ms
Wall time: 114 ms


NodeWithScore(node=TextNode(id_='98d21f17-e58d-481e-b72b-3334c1760e75', embedding=None, metadata={'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', 'publish_time': '2024-06-18 15:34:33', 'author': '林琛力', 'images': []}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明', 'publish_time': '2024-06-18 15:34:33', 'author': '林琛力', 'images': []}, hash='9218ed62df3afe0b78f534f578c7a9fc61f51997670d9a6988fd8a4ed05bc14e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='95e2eab9-aca6-4989-9123-071a62600954', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='432b8c5c1bcbfb0c4d81d195c93e616aa69a3994fb48949b746e68aceaf6e0bb')}, text='据中国驻菲律宾大使馆微信公众号消息,中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明。\n一、问:近日,英国路透社报道披露,新冠疫情期间美国军方为了遏制中国影响力,发动秘密心理战,使用社交媒体账号假冒菲律宾人散播中国疫苗和抗疫物资不安全的虚假信息,煽动菲民众对中国疫苗的不信任和恐惧,导

In [27]:
retrieved_nodes[0].metadata['title']

'中国驻菲律宾使馆发言人就美国军方发动秘密心理战抹黑中国疫苗发表声明'